In [1]:
import alpaca_trade_api as tradeapi
import threading
import time
import datetime
# config.py contains my keys
from config import *
import numpy as np
import pandas as pd
import edhec_risk_kit as erk
import bs4 as bs
import pickle
import requests
import yfinance as yf
APCA_API_BASE_URL = "https://paper-api.alpaca.markets"
%load_ext autoreload
%autoreload 2

### Get the tickers from the S&P 500

In [2]:
sp500_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tickers = sp500_data[0]['Ticker Symbol'].tolist()
# drop BKR, BRK.B and BF.B
# Alpaca doesn't like BKR
# Yahoo finance doesn't like BRK.B and BF.B
tickers = [x for x in tickers if x != 'BKR']
tickers = [x for x in tickers if x != 'BRK.B']
tickers = [x for x in tickers if x != 'BF.B']
tickers = ' '.join(tickers)

### Download stock close price data for 2019

In [3]:
data = yf.download(tickers, start='2019-01-01', end='2019-12-31')

[*********************100%***********************]  502 of 502 completed


### Calculate daily returns

In [4]:
rets = data['Adj Close'].pct_change()
# first drop columns where all of the values are NA
rets.dropna(axis=1, inplace=True, how='all')
# then drop the first row (where all the values are NA)
rets.dropna(axis=0, inplace=True, how='all')

### Calculate weights of the global minimum variance portfolio

In [ ]:
weights = np.round(erk.gmv(rets.cov()), 5)

### Connect to the API & retrieve the amount of cash I have in my paper account

In [ ]:
alpaca = tradeapi.REST(API_KEY, API_SECRET, APCA_API_BASE_URL, 'v2')
cash = int(float(alpaca.get_account().cash))

### Cancel any existing open orders

In [ ]:
orders = alpaca.list_orders(status='open')
for order in orders:
    alpaca.cancel_order(order.id)

### Calculate the dollar amount I want to buy of each stock

In [ ]:
dollar_amounts = weights*cash

In [ ]:
# function from https://github.com/alpacahq/alpaca-trade-api-python/blob/master/examples/long-short.py
# slightly altered
def submitOrder(qty, stock, side):
    if qty > 0:
        try: 
            alpaca.submit_order(stock, qty, side, 'market', 'day')
            print('Submitted order to ' + side + ' ' + str(qty) + ' shares(s) of ' + stock)
        except:
            print('Order failed to submit: ' + side + ' of ' + str(qty) + ' share(s) of ' + stock)
    else:
        print('Quantity for ' + stock + ' is zero b/c dollar_amount < share_price')

### Calculate quantity of shares of each stock that I'm buying & submit orders

In [ ]:
stocks = list(data['Adj Close'].columns)
for stock, amount in zip(stocks, dollar_amounts):
    try:
        # If the dollar amount that I want to buy of a stock is greater than $1,
        # attempt to retrieve the share price & submit an order
        if amount > 1:
            qty = int(amount//alpaca.get_barset(stock, 'minute', 1)[stock][0].c)
            submitOrder(qty, stock, 'buy')
    except:
        print("Couldn't pull stock data for " + stock + " from Alpaca")